<a href="https://colab.research.google.com/github/greygrease/tf-stuff/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdunction to NLP in Tensorlfow

NLP has the goal of deriving information our of natural language (text or speech)

Commom NLP problems are sequence to sequence problems (seq2seq)

## Get helper function

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-09-12 19:42:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-12 19:42:58 (62.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Import series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback,compare_historys, plot_loss_curves

## Get a text dataset
Kaggle's Introdunction to NLP dataset



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-09-12 19:43:02--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.1.208, 172.253.122.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.1.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-09-12 19:43:02 (127 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
# Unzip data

unzip_data('nlp_getting_started.zip')

## Visualize a text dataset

In [5]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [6]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
len(train_df)

7613

In [11]:
len(test_df)

3263

In [12]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index : random_index + 5].itertuples():
  print(row)
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n,{text}\n')
  print(f'----\n')

Pandas(Index=6675, text='@KristyLeeMusic brings her Alabama thunder back to the Attic SEPTEMBER 26! Tickets: https://t.co/B7ZwEVsrGO', target=0)
Target: 0 (not real disaster)
Text:
,@KristyLeeMusic brings her Alabama thunder back to the Attic SEPTEMBER 26! Tickets: https://t.co/B7ZwEVsrGO

----

Pandas(Index=4040, text='#NaturalDisasters As California fires rage the Forest Service sounds the alarm about rising wildfire costs: \x89Û_...  http://t.co/TQwrW3jQWo', target=1)
Target: 1 (real disaster)
Text:
,#NaturalDisasters As California fires rage the Forest Service sounds the alarm about rising wildfire costs: Û_...  http://t.co/TQwrW3jQWo

----

Pandas(Index=2230, text="@schelbertgeorg Thanks. I'm teaching an online class &amp; asking my students lots of questions like this. Sorry for the deluge of Ren. art!", target=0)
Target: 0 (not real disaster)
Text:
,@schelbertgeorg Thanks. I'm teaching an online class &amp; asking my students lots of questions like this. Sorry for the deluge of

### Split data into training and validation sets


In [14]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                              train_df_shuffled['target'].to_numpy(),
                                                              test_size = 0.1,
                                                              random_state = 42)

In [15]:
# Check the lenghts

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Convert text into numbers

Tokenisation - Direct mapping of tokens

Embedding - Create a matrix of feature vectors


### Text vectorisation

In [16]:
train_sentences[:3]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....'],
      dtype=object)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None, # limit on the no of tokens (uncommon words get replaced by <OOV>)
                                     standardize = 'lower_and_strip_punctuation', # lowers text and removes punctuation
                                     split = 'whitespace', #split where the white spaces exist
                                     ngrams = None, 
                                     output_mode = 'int',
                                    output_sequence_length = None, # how long the sequences should be
                                    # pad_to_max_tokens = True
                                    )

In [18]:
# Find the average number of tokens in thee trainint twee
ln = (sum([len(i.split()) for i in train_sentences]))/len(train_sentences)
round(ln)

15

In [19]:
# Setup text vectorisation variables

max_vocab_length = 10000
max_length = round(ln)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [20]:
# Fit the text vectoriser

text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it 

random_sentence = random.choice(train_sentences)
print(f'OG sent:\n {random_sentence}\n')

print(f'Vectorized sent:\n {text_vectorizer([random_sentence])}')

OG sent:
 California &amp;#039;monster&amp;#039; fire is 20% contained as 13000 are told to evacuate http://t.co/aPTAP6Yx1r

Vectorized sent:
 [[  90    1   42    9  693 3976   26 3220   22 1167    5  279    1    0
     0]]


In [23]:
text_vectorizer([random_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  90,    1,   42,    9,  693, 3976,   26, 3220,   22, 1167,    5,
         279,    1,    0,    0]])>

In [24]:
# Get the unique words in our vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()

words_in_vocab[:5]


['', '[UNK]', 'the', 'a', 'in']

In [25]:
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [26]:
len(words_in_vocab)

10000

In [27]:
words_in_vocab[-1]

'paddytomlinson1'

### Create an embedding representation of words

Parameters:

      input_dim = size of vocab
      output_dim = size of the output_sequence_length
      input_length = length of the seq


In [28]:
tf.random.set_seed(42)
from tensorflow.keras import layers

vocab_length = len(words_in_vocab)

embedding = layers.Embedding(input_dim = vocab_length,
                             output_dim = 128,
                             embeddings_initializer = 'uniform',
                             input_length = max_length)

In [29]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)

print(f'Original text:\n {random_sentence}\n')

print('After Embedding:')

# Embed the ranodm sent

sample_embed = embedding(text_vectorizer([random_sentence]))

print(sample_embed)

Original text:
 Black Eye 9: A space battle occurred at Star O784 involving 3 fleets totaling 3942 ships with 14 destroyed

After Embedding:
tf.Tensor(
[[[ 0.02947695 -0.04817411  0.00089605 ... -0.02419727 -0.01089432
    0.00691551]
  [ 0.01559405 -0.01473304  0.01601383 ...  0.03084493  0.00163232
   -0.02909597]
  [-0.02761239  0.04526105  0.0142337  ... -0.00329167 -0.01721014
    0.04758156]
  ...
  [-0.00987239 -0.00107161 -0.02644991 ...  0.0221944   0.03307087
    0.00479814]
  [-0.00584503  0.00410954  0.0016288  ...  0.02336507  0.01383555
   -0.04154969]
  [-0.01148288  0.04349462  0.04735816 ...  0.00710474  0.00304568
   -0.02277708]]], shape=(1, 15, 128), dtype=float32)


In [30]:
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02947695, -0.04817411,  0.00089605, ..., -0.02419727,
         -0.01089432,  0.00691551],
        [ 0.01559405, -0.01473304,  0.01601383, ...,  0.03084493,
          0.00163232, -0.02909597],
        [-0.02761239,  0.04526105,  0.0142337 , ..., -0.00329167,
         -0.01721014,  0.04758156],
        ...,
        [-0.00987239, -0.00107161, -0.02644991, ...,  0.0221944 ,
          0.03307087,  0.00479814],
        [-0.00584503,  0.00410954,  0.0016288 , ...,  0.02336507,
          0.01383555, -0.04154969],
        [-0.01148288,  0.04349462,  0.04735816, ...,  0.00710474,
          0.00304568, -0.02277708]]], dtype=float32)>

In [31]:
# Check out a singe token's embedding

sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.02947695, -0.04817411,  0.00089605,  0.03796096, -0.03667009,
       -0.04108192,  0.02550891,  0.02615688,  0.04612247,  0.0226249 ,
       -0.02899616,  0.01820681, -0.03055285, -0.03058353, -0.04816989,
       -0.0353957 , -0.01632122,  0.03934145,  0.03751752,  0.03563709,
        0.01391463,  0.02521173, -0.00596488,  0.01391793,  0.03608542,
       -0.00550788, -0.01646977,  0.03091994,  0.04451013,  0.02252548,
        0.0039793 ,  0.00377151,  0.02911669, -0.04814995,  0.04229248,
       -0.01693139,  0.03968037,  0.00580784, -0.04233699,  0.00043813,
       -0.03514503,  0.00915176,  0.02108878, -0.04366582,  0.0356187 ,
        0.0160802 ,  0.0107301 ,  0.0298891 ,  0.03625579, -0.0219695 ,
       -0.00113846, -0.04285799, -0.04635958, -0.0209532 ,  0.03133344,
        0.03243202,  0.01506353,  0.02538646, -0.02202332,  0.0316492 ,
        0.03748411,  0.0135189 ,  0.03964132, -0.03831694, -0.03797656,
        0.042319

In [32]:
random_sentence

'Black Eye 9: A space battle occurred at Star O784 involving 3 fleets totaling 3942 ships with 14 destroyed'

## Modelling a text dataet and running a series of experiments

    Model 0 : Naive Bayes (baseline)

    Model 1 : Feed Forward Neural Network (Dense)

    Model 2 : LSTM model (RNN)

    Model 3 : GRU model (RNN)

    Model 4 : Bidirectional LSTM model (RNN)

    Model 5 : 1D CNN

    Model 6:  Tensorflow Hub pretrained Feature Extractor (transfer learning)

    Model 7: Same as model 6 with 10% of training data

### Model 0 : Getting a baseline 


As with all machine learning experiments its important to create a baseline model for benchmarking

> Its common practice to use non_DL as baseline and use DL to see if we can improve upon it

In [33]:
from tensorflow.python.ops.gen_random_ops import Multinomial
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # turn our text to numbers
    ('clf', MultinomialNB())     # model the text
])

# Fit the pipe line to the training ata

model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [34]:
# Evaluate our baseline our model

baseline_score = model_0.score(val_sentences, val_labels)

In [35]:
print(f'Our baseline model achieves an accurayc of:{baseline_score*100}%')

Our baseline model achieves an accurayc of:79.26509186351706%


In [36]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [37]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

Compare our model using the following metrics:

>Accuracy

>Precision

>Recall

>F1-Score

In [38]:
# Function to eval 

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  # Accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Precison, Recall, F1-Score
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision' : model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results

In [39]:
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1 : A simple dense model

In [40]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Createa a directory to save TensorBoard lofs
SAVE_DIR = 'model_logs'

In [41]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = tf.string) # inputs are 1 dimensinal strting
x = text_vectorizer(inputs) # nums to vectors
x = embedding(x) 
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')


In [42]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [43]:
# Compile the model

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [44]:
# Fit the model

model_history = model_1.fit(x = train_sentences,
                          y = train_labels,
                          epochs = 5,
                          validation_data = (val_sentences, val_labels),
                          callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                   experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220912-194304
Epoch 1/5
215/215 [==============================] - 6s 25ms/step - loss: 0.6094 - accuracy: 0.6916 - val_loss: 0.5357 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 7s 32ms/step - loss: 0.4410 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 7s 31ms/step - loss: 0.3463 - accuracy: 0.8605 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 6s 30ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 7s 33ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4767 - val_accuracy: 0.7874


In [45]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 8ms/step - loss: 0.4767 - accuracy: 0.7874


[0.4766846001148224, 0.787401556968689]

In [46]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

array([0.40488204], dtype=float32)

In [47]:
model_1_pred_probs.shape

(762, 1)

In [48]:
# Convert model prediction probs to label formats

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [49]:
# Calcualte model 1 results
model_1_results = calculate_results(y_true = val_labels,
                            y_pred = model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

## Visualizing learned embeddings



In [50]:
# Get the vocab from the text vectorization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [51]:
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [52]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [53]:
# Get the wieght matrix of embedding layer

# These are the numerical representation of each token in our training data which have been 

embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights

array([[ 0.00073164,  0.01504797, -0.03425453, ..., -0.04403542,
        -0.01042278,  0.01876436],
       [ 0.04135864, -0.03945082, -0.03811938, ...,  0.00464735,
         0.03163553,  0.02928304],
       [ 0.00684034,  0.05363134, -0.00241555, ..., -0.07082174,
        -0.04750701,  0.01448254],
       ...,
       [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
         0.00308807,  0.02215792],
       [ 0.00692342,  0.05942352, -0.01975194, ..., -0.06199061,
        -0.01018394,  0.03510419],
       [-0.03723461,  0.06267188, -0.07451147, ..., -0.02367217,
        -0.08643329,  0.01742156]], dtype=float32)

In [54]:
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [55]:
# Download files from colab and upload to projector tool

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent neural network (RNN's)

> RNN's are useful for sequence data.

> The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of the later input.

> 

In [56]:
2784 *3*4

33408

### Model 2 : LSTM

LSTM = Long Short Term 

Structur of an RNN

    Input(text) -> Tokenize -> Embedding -> Layers(RNNs/Dense) -> Output (label probability)

In [57]:
# Create an LSTM Modle

from tensorflow.keras import layers

inputs = layers.Input(shape=(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.LSTM(64, return_sequences = True)(x) # when you're stacking RNN cells together you need to return sequences
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
x = layers.Dense(64, activation = 'relu')(x)
# print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

In [58]:
# Get a summary

model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

In [59]:
# Compile the model
model_2.compile(loss= 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics =['accuracy'])

In [60]:
# Fit the model

model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_2')])

Saving TensorBoard log files to: model_logs/model_2/20220912-194345
Epoch 1/5
215/215 [==============================] - 15s 51ms/step - loss: 0.2189 - accuracy: 0.9250 - val_loss: 0.5444 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 10s 45ms/step - loss: 0.1634 - accuracy: 0.9412 - val_loss: 0.6071 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 9s 42ms/step - loss: 0.1282 - accuracy: 0.9520 - val_loss: 0.7574 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 10s 45ms/step - loss: 0.1076 - accuracy: 0.9584 - val_loss: 0.9243 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 9s 44ms/step - loss: 0.0815 - accuracy: 0.9635 - val_loss: 0.9809 - val_accuracy: 0.7808


In [61]:
model_2_pred_prob = model_2.predict(val_sentences)

In [62]:
model_2_pred_prob[:10]

array([[2.3730695e-03],
       [5.7842922e-01],
       [9.9988604e-01],
       [5.8598995e-02],
       [2.6672602e-05],
       [9.9545348e-01],
       [9.3895888e-01],
       [9.9989402e-01],
       [9.9978161e-01],
       [3.9918748e-01]], dtype=float32)

In [63]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_prob))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [64]:
# Calcualte model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)

In [65]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [66]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [67]:
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

### Model_3: GRU

GRU - Gated Recurrent Unit
The GRU cell has similar features to an LSTM cell but has less parameters.

In [68]:
from tensorflow.keras import layers
inputs = layers.Input(shape =(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.LSTM(64, return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'model_3_GRU')

In [69]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [70]:
# Xomile the modle

model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics= ['accuracy'])

# Fit the model

model_3.history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_3_Gru')])

Saving TensorBoard log files to: model_logs/model_3_Gru/20220912-194512
Epoch 1/5
215/215 [==============================] - 12s 40ms/step - loss: 0.1620 - accuracy: 0.9377 - val_loss: 0.6392 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0887 - accuracy: 0.9672 - val_loss: 0.7860 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 7s 35ms/step - loss: 0.0717 - accuracy: 0.9721 - val_loss: 1.0992 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 7s 34ms/step - loss: 0.0630 - accuracy: 0.9739 - val_loss: 1.0155 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 7s 32ms/step - loss: 0.0519 - accuracy: 0.9758 - val_loss: 1.2545 - val_accuracy: 0.7782


In [71]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[4.7418475e-04],
       [7.2494292e-01],
       [9.9988055e-01],
       [9.6556336e-02],
       [6.1822859e-05],
       [9.9976516e-01],
       [9.6143854e-01],
       [9.9993181e-01],
       [9.9987465e-01],
       [9.8283720e-01]], dtype=float32)

In [72]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

In [73]:
model_3_results = calculate_results(y_true = val_labels,
                                    y_pred = model_3_preds)

In [74]:
model_3_results

{'accuracy': 77.82152230971128,
 'precision': 0.7791991586469295,
 'recall': 0.7782152230971129,
 'f1': 0.7766208088282468}

In [75]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [76]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [77]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right to left.

Bidrectional RNN goes from right to left aswell as left to right

In [78]:
# Bulid a bidirectional RNN in tensorflow

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 =  tf.keras.Model(inputs, outputs, name = 'model_4_bidirectional')

In [79]:
# Model 4 summary

model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [80]:
# Compile the model

model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


In [81]:
# Fit the model

model_4_history = model_4.fit(train_sentences, train_labels, epochs  =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'Model_4_Bidirectioanl')])

Saving TensorBoard log files to: model_logs/Model_4_Bidirectioanl/20220912-194553
Epoch 1/5
215/215 [==============================] - 17s 52ms/step - loss: 0.1096 - accuracy: 0.9692 - val_loss: 0.9659 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0519 - accuracy: 0.9762 - val_loss: 1.2894 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 9s 43ms/step - loss: 0.0458 - accuracy: 0.9791 - val_loss: 1.3640 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 10s 47ms/step - loss: 0.0431 - accuracy: 0.9806 - val_loss: 1.0320 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 9s 42ms/step - loss: 0.0458 - accuracy: 0.9796 - val_loss: 1.4890 - val_accuracy: 0.7664


In [82]:
# Make prediction 

model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[6.0182661e-05],
       [6.7251587e-01],
       [9.9995703e-01],
       [2.0951611e-01],
       [5.1625602e-06],
       [9.9471897e-01],
       [2.7759090e-01],
       [9.9997473e-01],
       [9.9995339e-01],
       [8.5341763e-01]], dtype=float32)

In [83]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [84]:
# Calculate the results of our bidirectional model

model_4_results = calculate_results(y_true = val_labels,
                                    y_pred = model_4_preds)

In [85]:
print( model_4_results, '\n', model_3_results, '\n', model_2_results, '\n', model_1_results, '\n', baseline_results)

{'accuracy': 76.64041994750657, 'precision': 0.7740790531264894, 'recall': 0.7664041994750657, 'f1': 0.7617181652846342} 
 {'accuracy': 77.82152230971128, 'precision': 0.7791991586469295, 'recall': 0.7782152230971129, 'f1': 0.7766208088282468} 
 {'accuracy': 78.08398950131233, 'precision': 0.7856875884556138, 'recall': 0.7808398950131233, 'f1': 0.777681907957685} 
 {'accuracy': 78.74015748031496, 'precision': 0.7914920592553047, 'recall': 0.7874015748031497, 'f1': 0.7846966492209201} 
 {'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Convolutional Neural Network for Text (and other type of sequences)

>Images are 2D (Conv2D)

>Text are 1D (Conv1D)

Typical Structure of Conv2D:

    Inputs(test) --> Tokenisatin --> Embedding --> Conv1D --> Pooling --> Outputs

### Model_5: Conv1D

####Brief Overview

In [88]:
# Test out our embedding layer, Conv1D layer and max pooling

embedding_text = embedding(text_vectorizer(['this is a test sentence'])) # Turn into Embedding
conv_1d = layers.Conv1D(filters = 64,
                        kernel_size = 5,
                        activation = 'relu',
                        padding = 'same')

In [89]:
conv_1d_output = conv_1d(embedding_text)
conv_1d_output[0]

<tf.Tensor: shape=(15, 64), dtype=float32, numpy=
array([[0.00000000e+00, 5.57034742e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.04869041e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.14954109e-02, 3.12996190e-03,
        3.07825636e-02, 0.00000000e+00, 0.00000000e+00, 5.29369116e-02,
        6.62494171e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.53746973e-02, 6.29911572e-02, 0.00000000e+00,
        1.23558166e-02, 5.34567423e-03, 1.39628137e-02, 0.00000000e+00,
        9.74638667e-03, 1.44570759e-02, 0.00000000e+00, 0.00000000e+00,
        3.22889984e-02, 1.20501956e-02, 0.00000000e+00, 9.12893005e-03,
        1.12925647e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.07065863e-02, 4.75994721e-02, 1.62930973e-02, 5.83052188e-02,
        2.89670601e-02, 0.00000000e+00, 5.32429926e-02, 0.00000000e+00,
        2.50303298e-02, 0.00000000e+00, 0.00000000e+00, 8.56353715e-02,
        0.0000

In [90]:
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # Get the most important feature
max_pool_output[0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([0.01157189, 0.06622932, 0.01503418, 0.00197518, 0.01809914,
       0.0304869 , 0.07663687, 0.08356027, 0.06075429, 0.04191777,
       0.03149541, 0.03628237, 0.03078256, 0.01940906, 0.01052775,
       0.0915478 , 0.05603867, 0.00068771, 0.02550259, 0.04267528,
       0.01305932, 0.09927102, 0.06299116, 0.00571592, 0.01235582,
       0.02436227, 0.01396281, 0.03040456, 0.0721628 , 0.09047666,
       0.03651918, 0.09263181, 0.032289  , 0.03211347, 0.00381231,
       0.04708476, 0.0640286 , 0.00612888, 0.04696149, 0.03027315,
       0.03070659, 0.07080074, 0.04900563, 0.06193311, 0.02896706,
       0.0412943 , 0.05324299, 0.02995514, 0.02503033, 0.04707731,
       0.04507045, 0.08563537, 0.03604575, 0.06201098, 0.03096467,
       0.09362949, 0.04047599, 0.01539855, 0.05810694, 0.05108888,
       0.08045309, 0.08559356, 0.06325339, 0.08091091], dtype=float32)>

In [91]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01735634, -0.08832304, -0.02602682, ..., -0.0124311 ,
         -0.06721227,  0.00288546],
        [-0.03448212,  0.01604004, -0.00287265, ..., -0.03159735,
         -0.01464332,  0.08202817],
        [-0.04435136, -0.04564711, -0.02855131, ..., -0.01368123,
          0.01099275,  0.05165091],
        ...,
        [-0.00810108, -0.010355  , -0.01404627, ..., -0.01834107,
         -0.01301464,  0.02730538],
        [-0.00810108, -0.010355  , -0.01404627, ..., -0.01834107,
         -0.01301464,  0.02730538],
        [-0.00810108, -0.010355  , -0.01404627, ..., -0.01834107,
         -0.01301464,  0.02730538]]], dtype=float32)>

In [92]:
conv_1d_output

<tf.Tensor: shape=(1, 15, 64), dtype=float32, numpy=
array([[[0.00000000e+00, 5.57034742e-03, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 3.04869041e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 3.14954109e-02, 3.12996190e-03,
         3.07825636e-02, 0.00000000e+00, 0.00000000e+00, 5.29369116e-02,
         6.62494171e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 3.53746973e-02, 6.29911572e-02, 0.00000000e+00,
         1.23558166e-02, 5.34567423e-03, 1.39628137e-02, 0.00000000e+00,
         9.74638667e-03, 1.44570759e-02, 0.00000000e+00, 0.00000000e+00,
         3.22889984e-02, 1.20501956e-02, 0.00000000e+00, 9.12893005e-03,
         1.12925647e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         3.07065863e-02, 4.75994721e-02, 1.62930973e-02, 5.83052188e-02,
         2.89670601e-02, 0.00000000e+00, 5.32429926e-02, 0.00000000e+00,
         2.50303298e-02, 0.00000000e+00, 0.00000000e+00, 8.56353715e-02

In [93]:
max_pool_output

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.01157189, 0.06622932, 0.01503418, 0.00197518, 0.01809914,
        0.0304869 , 0.07663687, 0.08356027, 0.06075429, 0.04191777,
        0.03149541, 0.03628237, 0.03078256, 0.01940906, 0.01052775,
        0.0915478 , 0.05603867, 0.00068771, 0.02550259, 0.04267528,
        0.01305932, 0.09927102, 0.06299116, 0.00571592, 0.01235582,
        0.02436227, 0.01396281, 0.03040456, 0.0721628 , 0.09047666,
        0.03651918, 0.09263181, 0.032289  , 0.03211347, 0.00381231,
        0.04708476, 0.0640286 , 0.00612888, 0.04696149, 0.03027315,
        0.03070659, 0.07080074, 0.04900563, 0.06193311, 0.02896706,
        0.0412943 , 0.05324299, 0.02995514, 0.02503033, 0.04707731,
        0.04507045, 0.08563537, 0.03604575, 0.06201098, 0.03096467,
        0.09362949, 0.04047599, 0.01539855, 0.05810694, 0.05108888,
        0.08045309, 0.08559356, 0.06325339, 0.08091091]], dtype=float32)>

###Building the model

In [94]:
# Create 1-dimenstioanl convolutional layer to model sequence

from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,
                  kernel_size = 5,
                  activation = 'relu',
                  padding = 'valid',
                  strides = 1)(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name = 'model_5_Conv1D')

# Compile the model
model_5.compile(loss ='binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [95]:
# Model 5 summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [96]:
# Fit the model

model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5 ,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20220912-195442
Epoch 1/5
215/215 [==============================] - 10s 35ms/step - loss: 0.1200 - accuracy: 0.9604 - val_loss: 0.8665 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0731 - accuracy: 0.9733 - val_loss: 1.0293 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 7s 32ms/step - loss: 0.0609 - accuracy: 0.9764 - val_loss: 1.1074 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 6s 28ms/step - loss: 0.0543 - accuracy: 0.9777 - val_loss: 1.1726 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0499 - accuracy: 0.9784 - val_loss: 1.2185 - val_accuracy: 0.7572


In [97]:
# Make some predictions with out Conv1D model

model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[9.8113835e-02],
       [6.6990888e-01],
       [9.9992985e-01],
       [8.5622787e-02],
       [1.5849207e-07],
       [9.9527365e-01],
       [9.7566652e-01],
       [9.9997991e-01],
       [9.9999928e-01],
       [8.7249631e-01]], dtype=float32)

In [98]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [99]:
model_5_results = calculate_results(y_true = val_labels,
                            y_pred = model_5_preds)

In [100]:
print(model_5_results, '\n', model_4_results, '\n', model_3_results, '\n', model_2_results, '\n', model_1_results, '\n', baseline_results)

{'accuracy': 75.7217847769029, 'precision': 0.7579616566971975, 'recall': 0.7572178477690289, 'f1': 0.7553466343855008} 
 {'accuracy': 76.64041994750657, 'precision': 0.7740790531264894, 'recall': 0.7664041994750657, 'f1': 0.7617181652846342} 
 {'accuracy': 77.82152230971128, 'precision': 0.7791991586469295, 'recall': 0.7782152230971129, 'f1': 0.7766208088282468} 
 {'accuracy': 78.08398950131233, 'precision': 0.7856875884556138, 'recall': 0.7808398950131233, 'f1': 0.777681907957685} 
 {'accuracy': 78.74015748031496, 'precision': 0.7914920592553047, 'recall': 0.7874015748031497, 'f1': 0.7846966492209201} 
 {'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Model_6 : TensorFlow Hub Pretrained Sentence Encoder

In [101]:
import tensorflow_hub as hub
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embed_samples = embed([sample_sentence, 'When you can the universal sentence encoder on a sentence, it turns it into numbers'])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157025  0.02485911  0.02878051 -0.012715    0.03971541  0.08827761
  0.02680988  0.05589838 -0.01068731 -0.00597293  0.00639321 -0.01819516
  0.00030816  0.09105889  0.05874645 -0.03180629  0.01512474 -0.05162925
  0.00991366 -0.06865345 -0.04209306  0.0267898   0.03011009  0.00321065
 -0.00337968 -0.04787356  0.0226672  -0.00985927 -0.04063615 -0.01292093
 -0.04666382  0.05630299 -0.03949255  0.00517682  0.02495827 -0.07014439
  0.0287151   0.0494768  -0.00633978 -0.08960193  0.02807119 -0.00808364
 -0.01360601  0.05998649 -0.10361788 -0.05195372  0.00232958 -0.02332531
 -0.03758106  0.03327729], shape=(50,), dtype=float32)


In [102]:
embed_samples

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157025,  0.02485911,  0.02878051, ..., -0.00186124,
         0.02315822, -0.01485021],
       [ 0.03485873, -0.08845595, -0.01677879, ..., -0.02750709,
         0.03230235, -0.00820085]], dtype=float32)>

In [103]:
# Create a keras layer using the USE Pretrained layer form tensorflow hub

sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                       input_shape = [],
                                       dtype = tf.string,
                                       name = 'USE')


In [104]:
# Create model using the Sequential API:

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = 'relu'),
    layers.Dense(1, activation = 'sigmoid')
], name = 'model_6_USE')

In [105]:
# Compile the model

model_6.compile(loss = 'binary_crossentropy',
            optimizer = tf.keras.optimizers.Adam(),
            metrics = ['accuracy'])

In [106]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [107]:
# Train a classifer on top of USE Pretrained embeddings

model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'tf_hub_sent_encoder')])

Saving TensorBoard log files to: model_logs/tf_hub_sent_encoder/20220912-195544
Epoch 1/5
215/215 [==============================] - 12s 30ms/step - loss: 0.5033 - accuracy: 0.7838 - val_loss: 0.4466 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4142 - accuracy: 0.8142 - val_loss: 0.4350 - val_accuracy: 0.8110
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3994 - accuracy: 0.8222 - val_loss: 0.4308 - val_accuracy: 0.8123
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3920 - accuracy: 0.8269 - val_loss: 0.4265 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3858 - accuracy: 0.8291 - val_loss: 0.4281 - val_accuracy: 0.8136


In [108]:
# Make predictions with USE tf hub models

model_6_pred_probs = model_6.predict(val_sentences)

In [109]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

In [110]:
# Calculate model 6 performance

model_6_results = calculate_results(y_true = val_labels,
                                    y_pred = model_6_preds)

In [111]:
print(model_6_results, '\n', model_5_results, '\n', model_4_results, '\n', model_3_results, '\n', model_2_results, '\n', model_1_results, '\n', baseline_results)

{'accuracy': 81.36482939632546, 'precision': 0.8160661319598898, 'recall': 0.8136482939632546, 'f1': 0.8120618868299143} 
 {'accuracy': 75.7217847769029, 'precision': 0.7579616566971975, 'recall': 0.7572178477690289, 'f1': 0.7553466343855008} 
 {'accuracy': 76.64041994750657, 'precision': 0.7740790531264894, 'recall': 0.7664041994750657, 'f1': 0.7617181652846342} 
 {'accuracy': 77.82152230971128, 'precision': 0.7791991586469295, 'recall': 0.7782152230971129, 'f1': 0.7766208088282468} 
 {'accuracy': 78.08398950131233, 'precision': 0.7856875884556138, 'recall': 0.7808398950131233, 'f1': 0.777681907957685} 
 {'accuracy': 78.74015748031496, 'precision': 0.7914920592553047, 'recall': 0.7874015748031497, 'f1': 0.7846966492209201} 
 {'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Model_7 : TF Hub pretrained but with 10% of training data

Transfer Learning really helps when you dont have a large dataset

In [135]:
# Create subsets of 10% of the training data (SPLITTING DATA LIKE THIS CAUSES DATA lEAKGAE)

# train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state =42)
# train_10_percent.head()

In [136]:
train_10_percent_split = int(0.1 * len(train_sentences))
train_sents_10_percent = train_sentences[:train_10_percent_split]

train_labels_10_percent = int(0.1 * len(train_labels))
train_labels_10_percent = train_labels[:train_labels_10_percent]


In [137]:
#
# train_sents_10_percent = train_10_percent['text'].to_list()
# train_labels_10_percent = train_10_percent['target'].to_list()

In [138]:
# train_10_percent['target'].value_counts()

In [139]:
# train_df_shuffled['target'].value_counts()

To recreate a model same as the previous model you created you can use the tf.keras clone model

In [140]:
# Lets build a model the same as model_6

model_7 = tf.keras.models.clone_model(model_6)

# Compile the model

model_7.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [141]:
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [142]:
# Fit the model
model_7.fit(train_sents_10_percent, train_labels_10_percent,
            validation_data = (val_sentences, val_labels),
            epochs = 5,
            callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                     'tf_hub_model_7_10%')])

Saving TensorBoard log files to: model_logs/tf_hub_model_7_10%/20220912-200911
Epoch 1/5
22/22 [==============================] - 4s 62ms/step - loss: 0.6713 - accuracy: 0.6788 - val_loss: 0.6512 - val_accuracy: 0.7283
Epoch 2/5
22/22 [==============================] - 1s 30ms/step - loss: 0.6032 - accuracy: 0.8029 - val_loss: 0.5941 - val_accuracy: 0.7625
Epoch 3/5
22/22 [==============================] - 1s 29ms/step - loss: 0.5263 - accuracy: 0.8204 - val_loss: 0.5377 - val_accuracy: 0.7703
Epoch 4/5
22/22 [==============================] - 1s 27ms/step - loss: 0.4638 - accuracy: 0.8204 - val_loss: 0.5064 - val_accuracy: 0.7730
Epoch 5/5
22/22 [==============================] - 1s 27ms/step - loss: 0.4221 - accuracy: 0.8263 - val_loss: 0.4902 - val_accuracy: 0.7808


In [143]:
# Make predictions with the models trained on 10% of the data

model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))

In [145]:
# Eval moddel 7

model_7_results = calculate_results(y_true = val_labels,
                                    y_pred = model_7_preds)
model_7_results

{'accuracy': 78.08398950131233,
 'precision': 0.7820967818762068,
 'recall': 0.7808398950131233,
 'f1': 0.7791507456344792}

## Comparing the performnace of each of our models

In [ ]:
# Combine model results into a DataFrame

all_model_results = pd.DataFrame({'baseline': baseline_results,
                                  '1_simple_dense': model_1_results,
                                  '2_LSTM': model_2_results,
                                  '3_GRU': model_3_results,
                                  ''})